In [1]:
from os import chdir
from google.colab import drive

drive.mount("/content/drive", force_remount=True)
chdir("/content/drive/MyDrive/Eliott/files/")

!pip install tweet-preprocessor
%pip install ipython-autotime
%pip install optuna

Mounted at /content/drive
     |████████████████████████████████| 308 kB 33.7 MB/s 
     |████████████████████████████████| 210 kB 74.9 MB/s 
     |████████████████████████████████| 81 kB 12.8 MB/s 
     |████████████████████████████████| 78 kB 4.6 MB/s 
     |████████████████████████████████| 113 kB 89.8 MB/s 
     |████████████████████████████████| 49 kB 8.5 MB/s 
     |████████████████████████████████| 146 kB 63.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=79688fb4d6108f0846c34f43e5d80cc0a473d1f3fde49c526e3da0ac5c925c66
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [2]:
# IMPORTS

import preprocessing as pp
import hyperparameters as hp
import RNN as rnn
import optimising as opt
import json
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import VotingClassifier



%load_ext autoreload
%autoreload 2


#
%load_ext autotime


time: 645 µs (started: 2022-04-14 14:40:17 +00:00)


In [3]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
X_train, y_train = pp.prepare_dataframe(file_name_train,lemmatising=None, original_class=False)
X_test, y_test = pp.prepare_dataframe(file_name_test,lemmatising=None, original_class=False)

time: 11.5 s (started: 2022-04-14 14:40:17 +00:00)


In [ ]:
X_train[0]

'and and'

time: 5.88 ms (started: 2022-04-12 23:56:42 +00:00)


On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys

- 35525 mots sans lemmatisation
- 30794 avec lemmatisation

In [ ]:
pipeline_sgd = hp.pipeline_sgd
pipeline_gb = hp.pipeline_gb
pipeline_rf = hp.pipeline_rf
pipeline_lr = hp.pipeline_lr
pipeline_per = hp.pipeline_per
pipeline_svc = hp.pipeline_svc

time: 2.59 ms (started: 2022-04-12 23:50:47 +00:00)


# Recherche d'Hyperparamètres

### Saving Hyperparameters in JSON

We will first load old weights and updates only if needed. Then we can use the model with best params in the end.

In [ ]:
with open('data.json') as json_file:
    dico = json.load(json_file)

time: 0 ns (started: 2022-04-08 12:04:37 -04:00)


### SGD

In [ ]:
%%time
grid_sgd = hp.grid_Search("SGD", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 720 candidates, totalling 3600 fits
Best Score:  0.8759385959288517
Best Params:  {'clf__alpha': 1.2000000000000002e-05, 'clf__penalty': 'l1', 'tfidf__max_df': 0.25, 'tfidf__min_df': 1, 'tfidf__ngram_range': (1, 1)}
CPU times: user 1min 41s, sys: 10.3 s, total: 1min 51s
Wall time: 29min 23s
time: 29min 23s (started: 2022-04-08 00:35:38 -04:00)


In [ ]:
hp.if_Save("SGD", dico, grid_sgd)

#### No new best score for  SGD
time: 1.22 ms (started: 2022-04-08 01:05:02 -04:00)


### Gradient boosting

In [ ]:
%%time
grid_gb = hp.grid_Search("GB", X_train=X_train, y_train=y_train, subset=1000) 

Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best Score:  0.608
Best Params:  {'clf__learning_rate': 0.8, 'clf__n_estimators': 400, 'tfidf__max_df': 1.0, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1)}
CPU times: user 5.43 s, sys: 44.3 ms, total: 5.47 s
Wall time: 6min 55s
time: 6min 55s (started: 2022-04-08 01:05:02 -04:00)


In [ ]:
hp.if_Save("GB", dico, grid_gb)

#### No new best score for  GB
time: 1.13 ms (started: 2022-04-08 01:11:58 -04:00)


### Random Forest

In [ ]:
%%time
grid_rf = hp.grid_Search("RF", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


KeyboardInterrupt: 

time: 23min 13s (started: 2022-04-08 02:19:33 -04:00)


In [ ]:
hp.if_Save('RF', dico, grid_rf)

### Logistic Regression

In [ ]:
%%time
grid_lr = hp.grid_Search("LR", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Score:  0.8230677619122002
Best Params:  {'clf__C': 10, 'clf__tol': 1e-12}
CPU times: user 1min 20s, sys: 2min 15s, total: 3min 35s
Wall time: 36min 42s


In [ ]:
hp.if_Save('LR', dico, grid_lr)

### Perceptron

In [ ]:
%%time
grid_per = hp.grid_Search("PER", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 300 candidates, totalling 1500 fits
Best Score:  0.818353925564533
Best Params:  {'clf__alpha': 3.04e-06, 'clf__penalty': 'l1'}
CPU times: user 1min 10s, sys: 7.51 s, total: 1min 17s
Wall time: 16min 16s


In [ ]:
hp.if_Save('PER', dico, grid_per)

#### Observation 

La regression logistique n'est qu'un perceptron avec une sigmoid en fonction d'activation.
On voit que la Regression Logistique a de meilleures performances à l'issue de la recherche d'hyperparamètres mais pas de loin. Par ailleurs le temps d'entrainement est considérablement plus élevé pour la regression logistique (du au calcul de l'exponentiel). Nous verrons par la suite quel modèle il est préférable de conserver. 

### LinearSVC

In [ ]:
%%time
grid_svc = hp.grid_Search("SVC", X_train=X_train, y_train=y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/svm/_classes.py", line 272, in fit
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sk

Best Score:  0.868916566574562
Best Params:  {'clf__dual': False, 'clf__loss': 'squared_hinge', 'clf__penalty': 'l1'}
CPU times: user 11.5 s, sys: 524 ms, total: 12.1 s
Wall time: 1min 7s


In [ ]:
hp.if_Save('SVC', dico, grid_svc)

### Charging the best parameters for training

In [ ]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)

data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']

time: 210 ms (started: 2022-04-12 23:50:55 +00:00)


### Training on all training data and testing on test data

In [ ]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  \n")
    print('------------------------------------------------------\n')
    print('\tClassification Report Train : \n\n', classification_report(y_train, predictions_train, target_names=['Negative','Neutral' ,'Positive']))
    print('------------------------------------------------------\n')
    print('\tClassification Report Test:\n\n',classification_report(y_test,predictions_test, target_names=['Negative', 'Neutral', 'Positive']))
    print('------------------------------------------------------\n')


time: 5.59 ms (started: 2022-04-12 23:51:00 +00:00)


### SGD

In [ ]:
data_sgd

{'best_score': 0.8769590395125022,
 'params': {'clf__alpha': 1.2000000000000002e-05,
  'clf__penalty': 'l1',
  'tfidf__max_df': 0.75,
  'tfidf__min_df': 1,
  'tfidf__ngram_range': [1, 1]}}

time: 3.94 ms (started: 2022-04-12 23:51:03 +00:00)


In [ ]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**(data_sgd["params"]),clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.75, ngram_range=[1, 1])),
                ('clf',
                 SGDClassifier(alpha=1.2000000000000002e-05, n_jobs=-1,
                               penalty='l1'))])

time: 9.52 ms (started: 2022-04-12 23:51:05 +00:00)


In [ ]:
%%time
"""
Si on a fait la recherche avant, on prend le best_model du gridSearchCv,
Sinon, on reconstruit le modèle avec les best paramètres et on le fit à toutes les données d'entrainement. 
"""

model_sgd_best.fit(X_train,y_train)


CPU times: user 2.6 s, sys: 282 ms, total: 2.88 s
Wall time: 1.77 s
time: 1.77 s (started: 2022-04-12 23:51:07 +00:00)


In [ ]:
testing(model_sgd_best)

train_accuracy : 0.9344461452486819 
test_accuracy : 0.8767772511848341  

------------------------------------------------------

	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.93      0.93     15398
     Neutral       0.93      0.90      0.91      7713
    Positive       0.94      0.95      0.94     18046

    accuracy                           0.93     41157
   macro avg       0.93      0.93      0.93     41157
weighted avg       0.93      0.93      0.93     41157

------------------------------------------------------

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.89      0.87      0.88      1633
     Neutral       0.85      0.84      0.84       619
    Positive       0.88      0.90      0.89      1546

    accuracy                           0.88      3798
   macro avg       0.87      0.87      0.87      3798
weighted avg       0.88      0.88      0.

### Random Forest

In [ ]:
model_rf_best = pipeline_rf
model_rf_best.set_params(**(data_rf["params"]),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 5.06 ms (started: 2022-04-12 19:35:30 -04:00)


In [ ]:
%%time

model_rf_best.fit(X_train,y_train)

CPU times: user 19.2 s, sys: 91.2 ms, total: 19.3 s
Wall time: 1.9 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

time: 1.9 s (started: 2022-04-12 19:35:31 -04:00)


In [ ]:
testing(model_rf_best)

train_accuracy : 0.8655375298490928 
test_accuracy : 0.6911064055055585  


	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.90      0.90      0.90     15317
     Neutral       0.90      0.58      0.71      7392
    Positive       0.83      0.96      0.89     17912

    accuracy                           0.87     40621
   macro avg       0.88      0.81      0.83     40621
weighted avg       0.87      0.87      0.86     40621

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.76      0.66      0.71      1631
     Neutral       0.73      0.39      0.51       604
    Positive       0.64      0.84      0.73      1543

    accuracy                           0.69      3778
   macro avg       0.71      0.63      0.65      3778
weighted avg       0.70      0.69      0.68      3778

time: 842 ms (started: 2022-04-12 19:35:36 -04:00)


### Gradient Boosting 

In [ ]:
model_gb_best = pipeline_gb
model_gb_best.set_params(**(data_gb["params"]))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

time: 5.05 ms (started: 2022-04-12 19:40:08 -04:00)


In [ ]:
%%time
model_gb_best.fit(X_train,y_train)

KeyboardInterrupt: 

time: 12 s (started: 2022-04-12 19:40:09 -04:00)


In [ ]:
testing(model_gb_best)

train_accuracy : 0.9979104405082975 
test_accuracy : 0.8457082675092154  


### Logistic Regression

In [ ]:
model_lr_best = pipeline_lr
model_lr_best.set_params(**(data_lr['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 4.29 ms (started: 2022-04-12 19:36:51 -04:00)


In [ ]:
%%time

model_lr_best.fit(X_train,y_train)

CPU times: user 561 ms, sys: 207 ms, total: 768 ms
Wall time: 12.5 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

time: 12.5 s (started: 2022-04-12 19:36:52 -04:00)


In [ ]:
testing(model_lr_best)

train_accuracy : 0.9775239408187884 
test_accuracy : 0.8213340391741663  


	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.97      0.98      0.98     15317
     Neutral       0.99      0.94      0.97      7392
    Positive       0.98      0.99      0.98     17912

    accuracy                           0.98     40621
   macro avg       0.98      0.97      0.98     40621
weighted avg       0.98      0.98      0.98     40621

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.85      0.82      0.83      1631
     Neutral       0.72      0.72      0.72       604
    Positive       0.83      0.86      0.85      1543

    accuracy                           0.82      3778
   macro avg       0.80      0.80      0.80      3778
weighted avg       0.82      0.82      0.82      3778

time: 596 ms (started: 2022-04-12 19:37:07 -04:00)


### Perceptron

In [ ]:
model_per_best = pipeline_per
model_per_best.set_params(**(data_per['params']),clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 4.84 ms (started: 2022-04-12 19:37:26 -04:00)


In [ ]:
%%time
model_per_best.fit(X_train,y_train)

CPU times: user 1.06 s, sys: 1.03 s, total: 2.1 s
Wall time: 732 ms


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

time: 737 ms (started: 2022-04-12 19:37:35 -04:00)


In [ ]:
testing(model_per_best)

train_accuracy : 0.904089017995618 
test_accuracy : 0.8194812069878242  


	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.91      0.91      0.91     15317
     Neutral       0.88      0.80      0.84      7392
    Positive       0.91      0.94      0.92     17912

    accuracy                           0.90     40621
   macro avg       0.90      0.88      0.89     40621
weighted avg       0.90      0.90      0.90     40621

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.85      0.82      0.84      1631
     Neutral       0.72      0.69      0.70       604
    Positive       0.82      0.87      0.84      1543

    accuracy                           0.82      3778
   macro avg       0.80      0.79      0.80      3778
weighted avg       0.82      0.82      0.82      3778

time: 580 ms (started: 2022-04-12 19:37:39 -04:00)


### SVC

In [ ]:
model_svc_best = pipeline_svc
model_svc_best.set_params(**(data_svc['params']))

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 3.33 ms (started: 2022-04-12 19:37:58 -04:00)


In [ ]:
%%time

model_svc_best.fit(X_train,y_train)

CPU times: user 4.78 s, sys: 17.5 ms, total: 4.8 s
Wall time: 4.8 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

time: 4.8 s (started: 2022-04-12 19:38:30 -04:00)


In [ ]:
testing(model_svc_best)

train_accuracy : 0.9460623815267964 
test_accuracy : 0.8642138697723664  


	Classification Report Train : 

               precision    recall  f1-score   support

    Negative       0.94      0.95      0.95     15317
     Neutral       0.95      0.90      0.93      7392
    Positive       0.95      0.96      0.95     17912

    accuracy                           0.95     40621
   macro avg       0.95      0.94      0.94     40621
weighted avg       0.95      0.95      0.95     40621

	Classification Report Test:

               precision    recall  f1-score   support

    Negative       0.88      0.86      0.87      1631
     Neutral       0.81      0.80      0.80       604
    Positive       0.87      0.89      0.88      1543

    accuracy                           0.86      3778
   macro avg       0.85      0.85      0.85      3778
weighted avg       0.86      0.86      0.86      3778

time: 584 ms (started: 2022-04-12 19:38:36 -04:00)


### Voting Classifier

In [ ]:
clf1 = model_sgd_best

clf2 = model_rf_best

clf3 = model_gb_best

clf4 = model_lr_best

clf5 = model_per_best

clf6 = model_svc_best


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


NameError: name 'model_gb_best' is not defined

time: 14.4 ms (started: 2022-04-07 22:45:31 -04:00)


In [ ]:
%%time

eclf1.fit(X_train,y_train)

CPU times: user 30min 13s, sys: 3.55 s, total: 30min 17s
Wall time: 30min 37s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.28206e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(criterion='entropy',
                                                                      max_features='log2',
                                                                      min_samples_split=32,
                                                                      n_estimators=200,
                                                                      n_j

In [ ]:
testing(eclf1)

train_accuracy : 0.9785698666083534 
test_accuracy : 0.8701948393891522  


In [ ]:
%%time

eclf2.fit(X_train,y_train)

NameError: name 'eclf2' is not defined

time: 113 ms (started: 2022-04-07 22:45:17 -04:00)


In [ ]:
testing(eclf2)

train_accuracy : 0.9609057997424496 
test_accuracy : 0.8654555028962612  


# Reccurent Network

In [ ]:
rnn_model = rnn.RNN(X_train, y_train, y_test, embedding_dim=25, units=210, dropout=0.2, n_neurons=50)

##### CREATING RNN #####
Vocabulary size: 38129

Example:

Sentence:
Cashier at grocery store was sharing his insights on To prove his credibility he commented "I'm in Civics class so I know what I'm talking about".

After tokenizing :
[1199, 15, 25, 18, 68, 1287, 242, 811, 10, 2, 2910, 242, 9777, 184, 9778, 220, 6, 18319, 1556, 53, 12, 124, 60, 220, 936, 58]

After padding :
[ 1199    15    25    18    68  1287   242   811    10     2  2910   242
  9777   184  9778   220     6 18319  1556    53    12   124    60   220
   936    58     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 63, 25)            953225    
                  

In [ ]:
rnn_model.train(epochs=2, batch_size=31) # Train the model

##### TRAINING #####
Epoch 1/2
1195/1195 [==============================] - 22s 15ms/step - loss: 0.1200 - accuracy: 0.7552 - val_loss: 0.0707 - val_accuracy: 0.8788
Epoch 2/2
1195/1195 [==============================] - 18s 15ms/step - loss: 0.0586 - accuracy: 0.9068 - val_loss: 0.0586 - val_accuracy: 0.8853
##### TRAINING COMPLETED #####


0.88532555103302

time: 39.3 s (started: 2022-04-13 07:09:09 +00:00)


In [ ]:
print(rnn_model.history.history['val_accuracy'][-1])

0.8892128467559814
time: 1.82 ms (started: 2022-04-13 01:55:23 +00:00)


In [ ]:
rnn_model.print_history() 

##### PRINTING HISTORY #####


##### PRINTING HISTORY COMPLETED #####


In [ ]:
rnn_model.test(X_test) # Test the model

##### TESTING #####
119/119 [==============================] - 1s 6ms/step - loss: 0.0667 - accuracy: 0.8707
Test loss: 0.06668345630168915
Test Accuracy: 0.8707214593887329
##### TESTING COMPLETED #####


(0.06668345630168915, 0.8707214593887329)

time: 1.8 s (started: 2022-04-13 01:56:49 +00:00)


In [ ]:
opt.optisearch(X_train=X_train, y_train=y_train, y_test=y_test, n_trials=60)

[I 2022-04-13 02:13:38,237] A new study created in memory with name: no-name-650f1651-8e76-48d2-9203-52dc6da0317e


{'epochs': 4, 'batch_size': 25, 'embedding_dim': 58, 'units': 269, 'dropout': 0.30000000000000004, 'n_neurons': 81}
Epoch 1/4
1482/1482 [==============================] - 31s 19ms/step - loss: 0.1135 - accuracy: 0.7737 - val_loss: 0.0655 - val_accuracy: 0.8793
Epoch 2/4
1482/1482 [==============================] - 27s 18ms/step - loss: 0.0559 - accuracy: 0.9105 - val_loss: 0.0563 - val_accuracy: 0.8975
Epoch 3/4
1482/1482 [==============================] - 27s 18ms/step - loss: 0.0367 - accuracy: 0.9485 - val_loss: 0.0543 - val_accuracy: 0.8970
Epoch 4/4
1482/1482 [==============================] - 27s 18ms/step - loss: 0.0286 - accuracy: 0.9621 - val_loss: 0.0575 - val_accuracy: 0.8904


[I 2022-04-13 02:15:32,991] Trial 0 finished with value: 0.8904275894165039 and parameters: {'epochs': 4, 'batch_size': 25, 'embedding_dim': 58, 'units': 269, 'dropout': 0.30000000000000004, 'n_neurons': 81}. Best is trial 0 with value: 0.8904275894165039.


{'epochs': 2, 'batch_size': 47, 'embedding_dim': 36, 'units': 251, 'dropout': 0.1, 'n_neurons': 93}
Epoch 1/2
789/789 [==============================] - 19s 20ms/step - loss: 0.1137 - accuracy: 0.7592 - val_loss: 0.0647 - val_accuracy: 0.8810
Epoch 2/2
789/789 [==============================] - 15s 19ms/step - loss: 0.0497 - accuracy: 0.9152 - val_loss: 0.0542 - val_accuracy: 0.8977


[I 2022-04-13 02:16:09,668] Trial 1 finished with value: 0.897716224193573 and parameters: {'epochs': 2, 'batch_size': 47, 'embedding_dim': 36, 'units': 251, 'dropout': 0.1, 'n_neurons': 93}. Best is trial 1 with value: 0.897716224193573.


...

{'epochs': 2, 'batch_size': 50, 'embedding_dim': 30, 'units': 287, 'dropout': 0.1, 'n_neurons': 91}
Epoch 1/2
741/741 [==============================] - 19s 21ms/step - loss: 0.1162 - accuracy: 0.7549 - val_loss: 0.0661 - val_accuracy: 0.8788
Epoch 2/2
741/741 [==============================] - 15s 20ms/step - loss: 0.0513 - accuracy: 0.9117 - val_loss: 0.0579 - val_accuracy: 0.8943


[I 2022-04-13 03:02:03,415] Trial 58 finished with value: 0.8943148851394653 and parameters: {'epochs': 2, 'batch_size': 50, 'embedding_dim': 30, 'units': 287, 'dropout': 0.1, 'n_neurons': 91}. Best is trial 32 with value: 0.8996598720550537.


{'epochs': 2, 'batch_size': 55, 'embedding_dim': 36, 'units': 503, 'dropout': 0.1, 'n_neurons': 50}
Epoch 1/2
674/674 [==============================] - 27s 35ms/step - loss: 0.1201 - accuracy: 0.7492 - val_loss: 0.0705 - val_accuracy: 0.8763
Epoch 2/2
674/674 [==============================] - 23s 34ms/step - loss: 0.0544 - accuracy: 0.9103 - val_loss: 0.0614 - val_accuracy: 0.8778


[I 2022-04-13 03:02:56,735] Trial 59 finished with value: 0.8777939677238464 and parameters: {'epochs': 2, 'batch_size': 55, 'embedding_dim': 36, 'units': 503, 'dropout': 0.1, 'n_neurons': 50}. Best is trial 32 with value: 0.8996598720550537.


Number of finished trials:  60
Best value: 0.8996598720550537
Best params: {'epochs': 2, 'batch_size': 35, 'embedding_dim': 38, 'units': 268, 'dropout': 0.1, 'n_neurons': 70}
time: 49min 18s (started: 2022-04-13 02:13:38 +00:00)


In [ ]:
opt.optisearch(X_train=X_train, y_train=y_train, y_test=y_test, n_trials=100)

[I 2022-04-13 04:20:40,008] A new study created in memory with name: no-name-57fab915-6d3e-4d2e-960a-1bc644ec0fe0


{'epochs': 3, 'batch_size': 24, 'embedding_dim': 24, 'units': 263, 'dropout': 0.4, 'n_neurons': 78}
Epoch 1/3
1544/1544 [==============================] - 27s 15ms/step - loss: 0.1240 - accuracy: 0.7501 - val_loss: 0.0704 - val_accuracy: 0.8669
Epoch 2/3
1544/1544 [==============================] - 23s 15ms/step - loss: 0.0637 - accuracy: 0.8986 - val_loss: 0.0579 - val_accuracy: 0.8938
Epoch 3/3
1544/1544 [==============================] - 22s 15ms/step - loss: 0.0442 - accuracy: 0.9385 - val_loss: 0.0568 - val_accuracy: 0.8921


[I 2022-04-13 04:21:55,089] Trial 0 finished with value: 0.8921282887458801 and parameters: {'epochs': 3, 'batch_size': 24, 'embedding_dim': 24, 'units': 263, 'dropout': 0.4, 'n_neurons': 78}. Best is trial 0 with value: 0.8921282887458801.


{'epochs': 2, 'batch_size': 28, 'embedding_dim': 26, 'units': 219, 'dropout': 0.30000000000000004, 'n_neurons': 115}
Epoch 1/2
1323/1323 [==============================] - 22s 14ms/step - loss: 0.1190 - accuracy: 0.7559 - val_loss: 0.0669 - val_accuracy: 0.8795
Epoch 2/2
1323/1323 [==============================] - 18s 13ms/step - loss: 0.0571 - accuracy: 0.9070 - val_loss: 0.0554 - val_accuracy: 0.8989


[I 2022-04-13 04:22:37,506] Trial 1 finished with value: 0.8989310264587402 and parameters: {'epochs': 2, 'batch_size': 28, 'embedding_dim': 26, 'units': 219, 'dropout': 0.30000000000000004, 'n_neurons': 115}. Best is trial 1 with value: 0.8989310264587402.


...

{'epochs': 3, 'batch_size': 30, 'embedding_dim': 25, 'units': 265, 'dropout': 0.1, 'n_neurons': 104}
Epoch 1/3
1235/1235 [==============================] - 26s 18ms/step - loss: 0.1092 - accuracy: 0.7731 - val_loss: 0.0680 - val_accuracy: 0.8744
Epoch 2/3
1235/1235 [==============================] - 22s 18ms/step - loss: 0.0508 - accuracy: 0.9126 - val_loss: 0.0570 - val_accuracy: 0.8890
Epoch 3/3
1235/1235 [==============================] - 22s 18ms/step - loss: 0.0320 - accuracy: 0.9493 - val_loss: 0.0633 - val_accuracy: 0.8761


[I 2022-04-13 05:47:19,283] Trial 98 finished with value: 0.8760932683944702 and parameters: {'epochs': 3, 'batch_size': 30, 'embedding_dim': 25, 'units': 265, 'dropout': 0.1, 'n_neurons': 104}. Best is trial 45 with value: 0.9020894169807434.


{'epochs': 2, 'batch_size': 31, 'embedding_dim': 20, 'units': 273, 'dropout': 0.30000000000000004, 'n_neurons': 96}
Epoch 1/2
1195/1195 [==============================] - 26s 20ms/step - loss: 0.1183 - accuracy: 0.7553 - val_loss: 0.0662 - val_accuracy: 0.8790
Epoch 2/2
1195/1195 [==============================] - 22s 19ms/step - loss: 0.0574 - accuracy: 0.9075 - val_loss: 0.0578 - val_accuracy: 0.8921


[I 2022-04-13 05:48:10,535] Trial 99 finished with value: 0.8921282887458801 and parameters: {'epochs': 2, 'batch_size': 31, 'embedding_dim': 20, 'units': 273, 'dropout': 0.30000000000000004, 'n_neurons': 96}. Best is trial 45 with value: 0.9020894169807434.


Number of finished trials:  100
Best value: 0.9020894169807434
Best params: {'epochs': 2, 'batch_size': 31, 'embedding_dim': 25, 'units': 210, 'dropout': 0.2, 'n_neurons': 50}
time: 1h 27min 30s (started: 2022-04-13 04:20:40 +00:00)


In [ ]:
opt.optisearch(X_train=X_train, y_train=y_train, y_test=y_test, n_trials=400)

[I 2022-04-13 07:11:59,741] A new study created in memory with name: no-name-54ad69b2-83b2-41a8-bb75-4cda4fc5c90b


{'epochs': 2, 'batch_size': 21, 'embedding_dim': 31, 'units': 296, 'dropout': 0.2, 'n_neurons': 54}
Epoch 1/2
1764/1764 [==============================] - 30s 15ms/step - loss: 0.1145 - accuracy: 0.7715 - val_loss: 0.0658 - val_accuracy: 0.8793
Epoch 2/2
1764/1764 [==============================] - 25s 14ms/step - loss: 0.0576 - accuracy: 0.9083 - val_loss: 0.0599 - val_accuracy: 0.8863


[I 2022-04-13 07:12:57,350] Trial 0 finished with value: 0.8862974047660828 and parameters: {'epochs': 2, 'batch_size': 21, 'embedding_dim': 31, 'units': 296, 'dropout': 0.2, 'n_neurons': 54}. Best is trial 0 with value: 0.8862974047660828.


{'epochs': 3, 'batch_size': 26, 'embedding_dim': 32, 'units': 300, 'dropout': 0.4, 'n_neurons': 68}
Epoch 1/3
1425/1425 [==============================] - 25s 16ms/step - loss: 0.1254 - accuracy: 0.7514 - val_loss: 0.0703 - val_accuracy: 0.8741
Epoch 2/3
1425/1425 [==============================] - 21s 15ms/step - loss: 0.0651 - accuracy: 0.8987 - val_loss: 0.0566 - val_accuracy: 0.8982
Epoch 3/3
1425/1425 [==============================] - 21s 15ms/step - loss: 0.0452 - accuracy: 0.9397 - val_loss: 0.0576 - val_accuracy: 0.8887


[I 2022-04-13 07:14:07,682] Trial 1 finished with value: 0.8887268900871277 and parameters: {'epochs': 3, 'batch_size': 26, 'embedding_dim': 32, 'units': 300, 'dropout': 0.4, 'n_neurons': 68}. Best is trial 1 with value: 0.8887268900871277.


...

{'epochs': 2, 'batch_size': 29, 'embedding_dim': 29, 'units': 263, 'dropout': 0.2, 'n_neurons': 66}
Epoch 1/2
1278/1278 [==============================] - 27s 19ms/step - loss: 0.1158 - accuracy: 0.7629 - val_loss: 0.0657 - val_accuracy: 0.8827
Epoch 2/2
1278/1278 [==============================] - 23s 18ms/step - loss: 0.0561 - accuracy: 0.9065 - val_loss: 0.0562 - val_accuracy: 0.8987


[I 2022-04-13 09:41:03,989] Trial 157 finished with value: 0.898688018321991 and parameters: {'epochs': 2, 'batch_size': 29, 'embedding_dim': 29, 'units': 263, 'dropout': 0.2, 'n_neurons': 66}. Best is trial 18 with value: 0.9040330648422241.


{'epochs': 2, 'batch_size': 29, 'embedding_dim': 30, 'units': 256, 'dropout': 0.2, 'n_neurons': 62}
Epoch 1/2
1278/1278 [==============================] - 21s 14ms/step - loss: 0.1183 - accuracy: 0.7559 - val_loss: 0.0768 - val_accuracy: 0.8499
Epoch 2/2
1278/1278 [==============================] - 17s 13ms/step - loss: 0.0558 - accuracy: 0.9097 - val_loss: 0.0566 - val_accuracy: 0.8936


[I 2022-04-13 09:41:44,141] Trial 158 finished with value: 0.8935859799385071 and parameters: {'epochs': 2, 'batch_size': 29, 'embedding_dim': 30, 'units': 256, 'dropout': 0.2, 'n_neurons': 62}. Best is trial 18 with value: 0.9040330648422241.


{'epochs': 2, 'batch_size': 28, 'embedding_dim': 28, 'units': 260, 'dropout': 0.2, 'n_neurons': 61}
Epoch 1/2
 749/1323 [===============>..............] - ETA: 7s - loss: 0.1406 - accuracy: 0.7000

In [ ]:
opt.optisearch(X_train=X_train, y_train=y_train, y_test=y_test, n_trials=60)

[I 2022-04-14 08:03:24,131] A new study created in memory with name: no-name-43840259-9a41-4312-84dd-161f652a801d


{'epochs': 4, 'batch_size': 63, 'embedding_dim': 48, 'units': 395, 'dropout': 0.30000000000000004, 'n_neurons': 88}
Epoch 1/4
588/588 [==============================] - 22s 23ms/step - loss: 0.1248 - accuracy: 0.7404 - val_loss: 0.0678 - val_accuracy: 0.8802
Epoch 2/4
588/588 [==============================] - 12s 21ms/step - loss: 0.0586 - accuracy: 0.9072 - val_loss: 0.0587 - val_accuracy: 0.8895
Epoch 3/4
588/588 [==============================] - 12s 21ms/step - loss: 0.0388 - accuracy: 0.9463 - val_loss: 0.0566 - val_accuracy: 0.8907
Epoch 4/4
588/588 [==============================] - 13s 21ms/step - loss: 0.0296 - accuracy: 0.9628 - val_loss: 0.0561 - val_accuracy: 0.8914
Epoch 1/4
588/588 [==============================] - 13s 22ms/step - loss: 0.0253 - accuracy: 0.9687 - val_loss: 0.0587 - val_accuracy: 0.8878
Epoch 2/4
588/588 [==============================] - 13s 22ms/step - loss: 0.0223 - accuracy: 0.9713 - val_loss: 0.0627 - val_accuracy: 0.8817
Epoch 3/4
588/588 [=======

[I 2022-04-14 08:08:33,158] Trial 0 finished with value: 0.8765306115150452 and parameters: {'epochs': 4, 'batch_size': 63, 'embedding_dim': 48, 'units': 395, 'dropout': 0.30000000000000004, 'n_neurons': 88}. Best is trial 0 with value: 0.8765306115150452.


{'epochs': 4, 'batch_size': 41, 'embedding_dim': 47, 'units': 133, 'dropout': 0.2, 'n_neurons': 62}
Epoch 1/4
904/904 [==============================] - 14s 12ms/step - loss: 0.1198 - accuracy: 0.7527 - val_loss: 0.0658 - val_accuracy: 0.8829
Epoch 2/4
904/904 [==============================] - 10s 11ms/step - loss: 0.0544 - accuracy: 0.9127 - val_loss: 0.0558 - val_accuracy: 0.8926
Epoch 3/4
904/904 [==============================] - 10s 11ms/step - loss: 0.0356 - accuracy: 0.9480 - val_loss: 0.0564 - val_accuracy: 0.8899
Epoch 4/4
904/904 [==============================] - 10s 11ms/step - loss: 0.0275 - accuracy: 0.9629 - val_loss: 0.0557 - val_accuracy: 0.8946
Epoch 1/4
904/904 [==============================] - 10s 11ms/step - loss: 0.0230 - accuracy: 0.9691 - val_loss: 0.0549 - val_accuracy: 0.8950
Epoch 2/4
904/904 [==============================] - 10s 11ms/step - loss: 0.0202 - accuracy: 0.9728 - val_loss: 0.0580 - val_accuracy: 0.8887
Epoch 3/4
904/904 [=======================

[I 2022-04-14 08:11:59,992] Trial 1 finished with value: 0.8804664731025695 and parameters: {'epochs': 4, 'batch_size': 41, 'embedding_dim': 47, 'units': 133, 'dropout': 0.2, 'n_neurons': 62}. Best is trial 1 with value: 0.8804664731025695.


...

{'epochs': 3, 'batch_size': 23, 'embedding_dim': 43, 'units': 290, 'dropout': 0.4, 'n_neurons': 98}
Epoch 1/3
1611/1611 [==============================] - 23s 12ms/step - loss: 0.1205 - accuracy: 0.7604 - val_loss: 0.0722 - val_accuracy: 0.8673
Epoch 2/3
1611/1611 [==============================] - 18s 11ms/step - loss: 0.0613 - accuracy: 0.9047 - val_loss: 0.0565 - val_accuracy: 0.8907
Epoch 3/3
1611/1611 [==============================] - 19s 12ms/step - loss: 0.0423 - accuracy: 0.9418 - val_loss: 0.0573 - val_accuracy: 0.8890
Epoch 1/3
1611/1611 [==============================] - 19s 12ms/step - loss: 0.0332 - accuracy: 0.9587 - val_loss: 0.0611 - val_accuracy: 0.8836
Epoch 2/3
1611/1611 [==============================] - 19s 12ms/step - loss: 0.0290 - accuracy: 0.9652 - val_loss: 0.0562 - val_accuracy: 0.8946
Epoch 3/3
1611/1611 [==============================] - 19s 12ms/step - loss: 0.0257 - accuracy: 0.9707 - val_loss: 0.0591 - val_accuracy: 0.8865
Epoch 1/3
1611/1611 [=========

[I 2022-04-14 10:39:29,324] Trial 58 finished with value: 0.8787657976150512 and parameters: {'epochs': 3, 'batch_size': 23, 'embedding_dim': 43, 'units': 290, 'dropout': 0.4, 'n_neurons': 98}. Best is trial 24 with value: 0.8905733704566956.


{'epochs': 2, 'batch_size': 59, 'embedding_dim': 48, 'units': 343, 'dropout': 0.1, 'n_neurons': 119}
Epoch 1/2
628/628 [==============================] - 17s 21ms/step - loss: 0.1147 - accuracy: 0.7591 - val_loss: 0.0689 - val_accuracy: 0.8676
Epoch 2/2
628/628 [==============================] - 12s 20ms/step - loss: 0.0499 - accuracy: 0.9159 - val_loss: 0.0539 - val_accuracy: 0.9001
Epoch 1/2
628/628 [==============================] - 13s 20ms/step - loss: 0.0311 - accuracy: 0.9522 - val_loss: 0.0539 - val_accuracy: 0.8938
Epoch 2/2
628/628 [==============================] - 12s 20ms/step - loss: 0.0237 - accuracy: 0.9654 - val_loss: 0.0533 - val_accuracy: 0.8967
Epoch 1/2
628/628 [==============================] - 12s 20ms/step - loss: 0.0202 - accuracy: 0.9703 - val_loss: 0.0572 - val_accuracy: 0.8909
Epoch 2/2
628/628 [==============================] - 12s 20ms/step - loss: 0.0173 - accuracy: 0.9738 - val_loss: 0.0595 - val_accuracy: 0.8916
Epoch 1/2
628/628 [======================

[I 2022-04-14 10:41:40,555] Trial 59 finished with value: 0.8896987438201904 and parameters: {'epochs': 2, 'batch_size': 59, 'embedding_dim': 48, 'units': 343, 'dropout': 0.1, 'n_neurons': 119}. Best is trial 24 with value: 0.8905733704566956.


Number of finished trials:  60
Best value: 0.8905733704566956
Best params: {'epochs': 2, 'batch_size': 60, 'embedding_dim': 53, 'units': 511, 'dropout': 0.1, 'n_neurons': 114}
time: 2h 38min 16s (started: 2022-04-14 08:03:24 +00:00)


In [ ]:
opt.optisearch(X_train=X_train, y_train=y_train, y_test=y_test, n_trials=60, CV=5)

[I 2022-04-14 14:41:18,292] A new study created in memory with name: no-name-8fa5f64d-ec14-4481-830e-2bcb79cf6555


{'epochs': 2, 'batch_size': 51, 'embedding_dim': 56, 'units': 455, 'dropout': 0.2, 'n_neurons': 55}
Epoch 1/2
727/727 [==============================] - 25s 24ms/step - loss: 0.1250 - accuracy: 0.7463 - val_loss: 0.0650 - val_accuracy: 0.8844
Epoch 2/2
727/727 [==============================] - 17s 23ms/step - loss: 0.0591 - accuracy: 0.9069 - val_loss: 0.0619 - val_accuracy: 0.8827
Epoch 1/2
727/727 [==============================] - 17s 23ms/step - loss: 0.0395 - accuracy: 0.9464 - val_loss: 0.0550 - val_accuracy: 0.8907
Epoch 2/2
727/727 [==============================] - 17s 24ms/step - loss: 0.0300 - accuracy: 0.9630 - val_loss: 0.0578 - val_accuracy: 0.8902
Epoch 1/2
727/727 [==============================] - 17s 24ms/step - loss: 0.0252 - accuracy: 0.9697 - val_loss: 0.0594 - val_accuracy: 0.8856
Epoch 2/2
727/727 [==============================] - 17s 24ms/step - loss: 0.0230 - accuracy: 0.9717 - val_loss: 0.0612 - val_accuracy: 0.8810
Epoch 1/2
727/727 [=======================

### Tests perso

In [ ]:
tests_perso = ["I think covid is the most horrible threat we ever faced",
                "I love covid, thanks to it I can see my family much more often and I don't have to comute as much",
                "I would love to come to your birthday party, but I got covid, I have to stay confined"
]

In [ ]:
predictions_out = eclf2.predict(tests_perso)


In [ ]:
predictions_out

array([-1,  1,  1])